## Importar Bibliotecas

In [15]:
import pickle
from gensim.models.wrappers import LdaMallet, ldamallet
from gensim.corpora.dictionary import Dictionary
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

NameError: name 'python' is not defined

## Carregar modelo

In [3]:
lda_mallet = LdaMallet.load("../../data/models/lda_model_400_stem_it_1000.mdl")
lda = ldamallet.malletmodel2ldamodel(lda_mallet)

In [13]:
num_topics = 400

## Funções 

In [4]:
def get_corpus_dict():
    corpus = pickle.load(open('../../data/corpus.pickle', 'rb'))
    dictionary = pickle.load((open('../../data/dictionary.pickle', 'rb')))
    
    return corpus, dictionary

In [5]:
def get_document_topics(lda, corpus, doc_id, max_topics):
    topics_df = pd.DataFrame()

    topics = sorted(lda[corpus][doc_id], key=lambda x: (x[1]), reverse=True)[0:max_topics]
        
    for topic in topics:
        topics_df = topics_df.append(
            pd.Series([
                topic[0], 
                str(round(topic[1]*100, 2)) + "%",
                ", ".join([word for (word, prop) in lda.show_topic(topic[0])])
            ]), ignore_index=True
        )
    
    topics_df.columns = ["Tópico Id", "Probabilidade", "Palavras"]
    
    return topics_df

In [6]:
def get_topic_documents(lda, corpus, topic_id, min_prob, num_documents):
    document_list = []

    for i, row in enumerate(lda[corpus]):
        for topic, prob in row:
            if topic == topic_id:
#             if topic == topic_id and prob > min_prob:
                document_list.append((i, prob))
                
#                 document_list.append((i, str(round(prob*100, 2)) + '%'))
    
    return sorted(document_list, key=lambda x: (x[1]), reverse=True)[0:num_documents]

In [7]:
def get_discurso(id):
    disc = pickle.load(open("../../data/discursos_raw/{}.pickle".format(id), 'rb'))

    print(disc["Conteudo"])

In [8]:
def show_topics(lda, num_topics):
    for index, topic in lda.show_topics(formatted=False, num_words= 10, num_topics=num_topics):
        print('{}\t{}'.format(index, ', '.join(a[0] for a in topic)))

In [9]:
def get_topics(lda, num_topics):
    res = []
    for index, topic in lda.show_topics(formatted=False, num_words= 10, num_topics=num_topics):
        res.append([a[0] for a in topic])
    
    return res

In [10]:
def search_topic(lda, word_input, num_topics):
    result = []
    for i, words in enumerate(get_topics(lda, num_topics)):
        for word in words:
            if word == word_input:
                result.append((i, ', '.join(words)))
    
    return result

In [11]:
corpus, dictionary = get_corpus_dict()

## Quais são os 200 Tópicos?

In [16]:
show_topics(lda, num_topics)

0	civil, defesa, tragedia, incendio, aconteceu, desastre, barragem, risco, nacional, ocorre
1	juros, taxa, economia, inflacao, alto, banco, baixa, central, aumento, brasil
2	medida, provisoria, governo, congresso, edital, urgencia, aprovado, materia, prazo, casa
3	aprovado, projeto, deputados, aprovacao, comissao, materia, autoria, regulamentacao, unanimidade, voto
4	trabalho, sindical, sindicato, central, movimento, nacional, entidades, luta, social, categoria
5	governo, oposicao, partido, base, lider, critica, alias, queria, tentativa, fazendo
6	anos, ultimos, durante, passado, trinta, longo, periodo, completamente, chegar, cinquenta
7	profissionais, tecnica, trabalho, curso, formacao, ensino, qualificacao, mercado, escola, qualificado
8	israel, mundo, terrorismo, palestina, iraque, arabes, conflitos, judeus, oriente, medio
9	aposentados, fatos, pensionistas, salario, previdenciario, aposentadoria, reajuste, minimo, previdencia, receber
10	economia, governo, fiscal, publico, corte, g

## Quais os tópicos que contém a palavra "violencia"?

In [18]:
pd.DataFrame(search_topic(lda, "violencia", num_topics), columns=["Tópico", "Palavras"])

,Tópico,Palavras
0,18,"seguranca, violencia, policia, crime, policial, droga, publico, sociedade, arma, pais"
1,35,"mulher, pessoa, direito, negro, crianca, trabalho, deficiencia, tambem, sociedade, violencia"


## Quais os tópicos do discurso número 100?

In [13]:
discurso_id = 24968

In [19]:
pd.set_option('display.max_columns', None)

get_document_topics(lda, corpus, discurso_id, 8)

,Tópico Id,Probabilidade,Palavras
0,35.0,22.99%,"mulher, pessoa, direito, negro, crianca, trabalho, deficiencia, tambem, sociedade, violencia"
1,48.0,17.03%,"saude, medico, hospital, doenca, atender, tratar, pessoa, publico, brasil, medicar"
2,26.0,10.5%,"senador, aparte, nobre, permitir, pronunciar, haver, pais, dever, querer, ouvir"
3,10.0,9.68%,"numero, dado, pais, brasileiro, crescer, pesquisa, milhoes, ultimo, populacao, indice"
4,47.0,7.73%,"situacao, problema, governo, haver, grave, solucao, sofrer, presidente, acontecer, viver"
5,27.0,6.49%,"questao, haver, querer, debate, algum, tambem, discutir, importante, outro, relacao"
6,29.0,5.07%,"programa, social, renda, familia, pessoa, pobre, pobreza, populacao, milhoes, fome"
7,0.0,2.65%,"vida, casa, senador, senado, homem, publico, politico, tambem, amigo, trabalho"


## Discurso 100:

In [15]:
print(get_discurso(discurso_id))

             O SR. ANIBAL DINIZ (Bloco/PT - AC. Pronuncia o seguinte discurso. Sem revisão do orador.) - Sr. Presidente, Senador Valdir Raupp; Senador Suplicy; telespectadores da TV Senado; ouvintes da Rádio Senado, chego aqui após ter participado de uma reunião bastante produtiva com o Governador do Acre, Tião Viana, que, hoje, esteve em audiência no Ministério dos Transportes, especificamente no DNIT. Participou também de uma audiência importante com a Ministra Miriam Belchior, juntamente com outros Governadores, para tratar especificamente do plano de investimentos do PAC e de algumas questões das obras que estão acontecendo no Estado, financiadas pelo PAC.             Também houve uma audiência no Ministério da Integração para tratar especificamente das unidades habitacionais que estão sendo autorizadas, a partir do projeto que o Governo do Estado apresentou, voltado para atender uma parte das famílias vítimas do alagamento em Rio Branco e outra parte integrante do grande programa 

## Quais são os principais documentos do tópico 78?

In [68]:
a = pd.DataFrame(get_topic_documents(lda, corpus, 31, 0.01, 20), columns=["Discurso", "Porcentagem"])


# filtered = pd.DataFrame(lda[corpus])


In [69]:
a

,Discurso,Porcentagem
0,83543,0.442959
1,28590,0.423104
2,51405,0.384633
3,80940,0.374920
4,83609,0.368371
5,24132,0.361475
6,28790,0.360800
7,28784,0.360564
8,28293,0.360066
9,53155,0.352225


In [70]:
print(get_discurso(83543))

 O SR. VALMIR AMARAL (PMDB - DF. Sem apanhamento taquigráfico.) - Sr. Presidente, Srªs e Srs. Senadores, como todos sabem, foi realizada em Brasília a 36ª edição do Festival de Brasília do Cinema Brasileiro. Foi uma grande oportunidade para atestarmos o vigor do cinema brasileiro, não obstante todos os percalços que tem sofrido ao longo de mais de um século de produção cinematográfica em terras tupiniquins. E o festival de Brasília é um dos mais tradicionais e conceituados do Brasil com reconhecimento de toda mídia especializada da América. A abertura solene ocorreu dia 18 de novembro e refletiu o prestígio desse festival. Mais de duas mil pessoas estiveram lá: público cinéfilo, diretores, fotógrafos, atores e atrizes, autoridades governamentais da área de cultura, deputados e senadores. No Teatro Nacional Cláudio Santoro, a majestosa música de nossa orquestra deu uma mostra do talento e da competência com que está sendo gerida a cultura em nossa cidade e em nosso País. Trata-se de um 